In [140]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest



In [194]:
data = pd.read_csv("../data/other_train.csv",index_col=0)

dict_array = []
string = '{"mean_glucose": 9999999,"std_glucose": 9999999,"kurtosis_glucose":9999999,"skewness_glucose":9999999,"mean_oxygen":9999999,"std_oxygen":9999999,"kurtosis_oxygen":9999999,"skewness_oxygen":9999999}'
data.medical_info.apply(lambda x: dict_array.append(json.loads(x.replace("'","\"").replace(":\"",":").replace("\",",",").replace("\"}","}") if isinstance(x,str) else string)))
medical_datas = pd.DataFrame(dict_array)

for item in medical_datas:
    data[item] = medical_datas[item].astype(float)
data = data.drop(columns=['medical_info'])
# print(data)
# data.info()


person = pd.read_csv("../data/personal_train.csv",index_col=0)
# print(person)
# person.info()

/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3932
Data columns (total 28 columns):
name                3933 non-null object
address             3933 non-null object
age                 3933 non-null float64
sex                 3933 non-null object
date_of_birth       3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3703 non-null object
pregnant            3931 non-null object
relationship        3932 non-null object
education           3933 non-null object
income              3932 non-null object
native-country      3872 non-null object
workclass           3341 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3932 non-null float64
class               3933 non-null float64
hours-per-week      3932 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3932 non-null float64
kurtosis_oxygen     3932

In [ ]:

data = data.replace({ 9999999 : np.nan },regex=False)
data = data.replace({'^[? ]*$' : np.nan},regex=True)
person = person.replace({'^[? ]*$' : np.nan},regex=True)

In [ ]:
numeric = data.select_dtypes(include=['float64'])
categoric = data.select_dtypes(include=['object'])
numeric['name'] = categoric['name']

numeric = numeric.groupby('name').aggregate('mean')
categoric = categoric.groupby('name').aggregate('first')

data = categoric.merge(numeric, on=['name'])

In [ ]:
merged_data = pd.merge(person, data, on=['name','address'], how='left')


In [ ]:
merged_data = merged_data.replace(['FALSE','F'],'f',regex=True)
merged_data = merged_data.replace(['TRUE','T'],'t',regex=True)

merged_data = merged_data.replace([' <=50K'],'<=50K',regex=True)
merged_data = merged_data.replace([' >50K'],'>50K',regex=True)


merged_data = merged_data.replace([' Other-relative'],' Other_relative',regex=True)
merged_data = merged_data.replace([' Not-in-family'],' Not_in_family',regex=True)
merged_data = merged_data.replace([' Own-child'],' Own_child',regex=True)


for i, row in merged_data.iterrows():
        
    date = parser.parse(merged_data.iloc[i]['date_of_birth'].split(' ')[0])
    day = date.day
    month = date.month
    year = date.year
    
    if (year > 2019):
        year = year - 100

    merged_data.at[i,'date_of_birth'] = str(day) + "/" + str(month) + "/" + str(year)
            
    if ((pd.isna(row['age'])) | (row['age'] > 115) | (row['age'] < 0)):
        merged_data.at[i,'age'] = 2019 - year

merged_data.info()

In [ ]:
# Remove outliers which can be probably not use
# print(merged_data['education-num'].value_counts())
# sns.distplot(merged_data['education-num'])

# ## Remove outliers from education-num
# # sns.boxplot(merged_data['education-num'])
# merged_data.loc[merged_data['education-num'] < 5,'education-num'] = np.NaN
# # sns.boxplot(merged_data['education-num'])
# 
# ## Remove outliers from 'capital-gain'
# # sns.boxplot(merged_data['capital-gain'])
# merged_data.loc[merged_data['capital-gain'] > 30000,'capital-gain'] = np.NaN
# # sns.boxplot(merged_data['capital-gain'])
# 
# ## Remove outliers from 'fnlwgt'
# # sns.boxplot(merged_data['fnlwgt'])
# merged_data.loc[merged_data['fnlwgt'] > 800000,'fnlwgt'] = np.NaN
# # sns.boxplot(merged_data['fnlwgt'])
# 
# ## Remove outliers from 'hours-per-week'
# # sns.boxplot(merged_data['hours-per-week'])
# merged_data.loc[(merged_data['hours-per-week'] > 57) | (merged_data['hours-per-week'] < 8),'hours-per-week'] = np.NaN
# # sns.boxplot(merged_data['hours-per-week'])

In [ ]:
# Remove outliers on floats datas

## Remove outliers from 'kurtosis_glucose'
column = 'kurtosis_glucose'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 6) | (merged_data[column] < -1)
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'mean_glucose'
column = 'mean_glucose'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 150)  
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'mean_oxygen'
column = 'mean_oxygen'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 120)  
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'kurtosis_oxygen'
column = 'kurtosis_oxygen'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 20)  
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'skewness_glucose'
column = 'skewness_glucose'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 40)  
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'skewness_oxygen'
column = 'skewness_oxygen'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 350)  
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'std_glucose'
column = 'std_glucose'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 62) 
merged_data.loc[mask ,column] = np.NaN

## Remove outliers from 'std_oxygen'
column = 'std_oxygen'
# sns.boxplot(merged_data[column])
# merged_data[column].hist()
mask = (merged_data[column] > 80)  
merged_data.loc[mask ,column] = np.NaN


In [64]:
# Divide datas into 3 category

old = merged_data
merged_data = merged_data.drop(columns=['class'])

numeric = merged_data.select_dtypes(include=['float64'])
numeric1 = numeric.iloc[:,:7].columns
print(numeric1)
numeric2 = numeric.iloc[:,7:].columns
print(numeric2)        
categoric = merged_data.select_dtypes(include=['object']).columns
print(categoric)

Index(['age', 'education-num', 'capital-gain', 'fnlwgt', 'hours-per-week',
       'capital-loss', 'kurtosis_glucose'],
      dtype='object')
Index(['kurtosis_oxygen', 'mean_glucose', 'mean_oxygen', 'skewness_glucose',
       'skewness_oxygen', 'std_glucose', 'std_oxygen'],
      dtype='object')
Index(['name', 'address', 'sex', 'date_of_birth', 'race', 'marital-status',
       'occupation', 'pregnant', 'relationship', 'education', 'income',
       'native-country', 'workclass'],
      dtype='object')


In [65]:
# Apply pipeline in category 

numeric_transformer1 = Pipeline(steps=[('imputer',MissForest(missing_values=np.NaN, max_iter=10, # Nahradenie nulovych hodnot pomocou MissForest
                                                             decreasing=False, copy=False)), 
                                      ('powerTransformer',PowerTransformer(method='yeo-johnson',
                                                                           copy=False)) # Transformuje dat aby boli viac z Gaussoveho rozdelenia
                                       ])
numeric_transformer2 = Pipeline(steps=[('imputer',KNNImputer(missing_values=np.NaN, n_neighbors=5, # Nahradenie nulovych hodnot pomocou KNN
                                                              weights='distance')), 
                                     # ('RobustScaler',StandardScaler())    # 
                                       ])
categoric_transormer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))]) # Nahradenie nulovych hodnot pomocou najcastejsich hodnot
                                      # ('onehot',OneHotEncoder(handle_unknown='ignore'))])
preproccesors = ColumnTransformer(transformers=[('Cat', categoric_transormer, categoric),
                                                ('Num1', numeric_transformer1, numeric1),
                                                ('Num2', numeric_transformer2, numeric2)
                                                ])


In [66]:
# Execute pipeline steps

pipeline = Pipeline(steps=[('preproccesors', preproccesors)])
pipeline_datas = pipeline.fit_transform(merged_data)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))


In [67]:
# Show mereged datas

merged_data = pd.DataFrame(pipeline_datas,columns=merged_data.columns)
print(merged_data)


                      name                                            address  \
0          Roscoe Bohannon  7183 Osborne Ways Apt. 651\r\nEast Andrew, OH ...   
1             Ernest Kline  391 Ball Road Suite 961\r\nflowersborough, IN ...   
2          Harold Hendriks     8702 Vincent Square\r\nNew Jerryfurt, CO 30614   
3           Randy Baptiste     2751 Harris Crossroad\r\nWest Ashley, CA 30311   
4          Anthony Colucci  904 Robert Cliffs Suite 186\r\nWest Kyle, CO 7...   
5             Ronald Lange        30973 Martinez Shores\r\nJameston, CA 70245   
6            Boyd Eiselein  29941 Christopher Curve Apt. 682\r\nRaychester...   
7            Raymond Smith  53487 Scott Extensions Apt. 824\r\nMccartytown...   
8            Harold Miller  8514 Elizabeth Crescent\r\nWest Joseland, GA 4...   
9     Charles Czachorowski  6798 Wagner Locks Suite 377\r\nLake Brenda, DC...   
10         Robert Grantham            822 Adams Plains\r\nNew Karen, AK 92114   
11                Joel Lam  